# Societe Generale Complaint Tracker

In [1]:
import pandas as panda

from sklearn.model_selection import learning_curve, train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, mean_absolute_error, classification_report, \
    confusion_matrix, f1_score, roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

from matplotlib import pyplot as plot
import seaborn as sns


from numpy import bincount, linspace, mean, std, arange, squeeze

import itertools, time, datetime

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

In [2]:
test_data_path ='dataset/test_all_merged_1.csv'
train_data_path = 'dataset/train_all_merged_1.csv'
sample_submission_path = 'dataset/sample_submission.csv'
test_complaint_reason_path = 'dataset/test_data_complaint_reason.csv'
train_complaint_reason_path = 'dataset/train_data_complaint_reason.csv'


In [3]:
train_data = panda.read_csv(train_data_path)
train_data.columns.tolist()

['Unnamed: 0',
 'complaint_id',
 'complaint_reason_x',
 'complaint_reason_encoded',
 'date_received',
 'transaction_type',
 'complaint_reason_y',
 'company_response',
 'date_sent_to_company',
 'complaint_status',
 'consumer_disputes',
 'consumer_complaint_summary',
 'days_passed_since_complaint_received',
 'days_since_complaint_sent_to_company',
 'Bank account or service',
 'Checking or savings account',
 'Consumer Loan',
 'Credit card',
 'Credit card or prepaid card',
 'Credit reporting',
 'Credit reporting, credit repair services, or other personal consumer reports',
 'Debt collection',
 'Money transfer, virtual currency, or money service',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Payday loan, title loan, or personal loan',
 'Prepaid card',
 'Student loan',
 'Vehicle loan or lease',
 'Virtual currency',
 '0',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',


In [8]:
## lets prepare our x data set values

columns_not_required = ['complaint_id',\
 'date_received',\
 'transaction_type',\
 'complaint_reason',\
 'company_response',\
 'date_sent_to_company',\
 'complaint_status',\
 'consumer_disputes',\
 'consumer_complaint_summary',\
                       'Unnamed: 0', 'complaint_reason.1', 'complaint_reason_x',\
'complaint_reason_y' ,'y']


x = train_data[[i for i in train_data.columns.tolist() if i not in columns_not_required]]

x.head()

,complaint_reason_encoded,days_passed_since_complaint_received,days_since_complaint_sent_to_company,Bank account or service,Checking or savings account,Consumer Loan,Credit card,Credit card or prepaid card,Credit reporting,"Credit reporting, credit repair services, or other personal consumer reports",...,Company believes complaint is the result of an isolated error,Company believes complaint relates to a discontinued policy or procedure,Company believes complaint represents an opportunity for improvement to better serve consumers,Company believes it acted appropriately as authorized by contract or law,Company believes the complaint is the result of a misunderstanding,Company can't verify or dispute the facts in the complaint,Company chooses not to provide a public response,Company disputes the facts presented in the complaint,Company has responded to the consumer and the CFPB and chooses not to provide a public response,encoded_consumer_disputes
0,8,1166,1166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,13,1293,1293,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,21,1354,1354,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,22,799,799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
4,24,843,843,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [14]:
y = train_data[['y']]
y.head()

,y
0,0
1,1
2,0
3,0
4,0


In [28]:
## its time to run my pipeline and classification models

from sklearn.ensemble import AdaBoostClassifier

## we will add more classifiers in round2
classifiers = [
#     Perceptron(random_state = 1),
#     LogisticRegression(random_state = 1),
    AdaBoostClassifier(RandomForestClassifier(random_state = 1, n_estimators = 10, criterion='entropy'), n_estimators = 400, learning_rate = 0.1),
#     DecisionTreeClassifier(random_state = 1, max_depth = 8, criterion='entropy')

#     RandomForestClassifier(random_state = 1),
#     KNeighborsClassifier(metric = 'minkowski'),
]


classifier_names = [
#             'perceptron',
#             'logisticregression',
            'decisiontreeclassifier',
#             'randomforestclassifier',
#             'kneighborsclassifier',
    
]

classifier_param_grid = [
            
#             {'perceptron__max_iter': [5,10,30], 'perceptron__eta0': [.1]},
#             {'logisticregression__C':[100,300, 500]},
#             {'decisiontreeclassifier__max_depth':[6,8,10],'decisiontreeclassifier__max_depth':[6,8,10],'decisiontreeclassifier__criterion':['gini',]},
    {}
#             {'randomforestclassifier__n_estimators':[6,8,12],'randomforestclassifier__criterion': ['gini',]} ,
#             {'kneighborsclassifier__n_neighbors':[4,6,10]},
    
]

In [10]:
class CodeTimer:
    
    """
        Utility custom contextual class for calculating the time 
        taken for a certain code block to execute
    
    """
    def __init__(self, name=None):
        self.name = " '"  + name + "'" if name else ''

    def __enter__(self):
        self.start = time.clock()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (time.clock() - self.start) * 1000.0
        time_taken = datetime.timedelta(milliseconds = self.took)
        print('Code block' + self.name + ' took(HH:MM:SS): ' + str(time_taken))

In [11]:

def plotLearningCurve(_x_train, _y_train, learning_model_pipeline,  k_fold = 10, training_sample_sizes = linspace(0.1,1.0,10), jobsInParallel = 1):
    
    training_size, training_score, testing_score = learning_curve(estimator = learning_model_pipeline, \
                                                                X = _x_train, \
                                                                y = _y_train, \
                                                                train_sizes = training_sample_sizes, \
                                                                cv = k_fold, \
                                                                n_jobs = jobsInParallel) 


    training_mean = mean(training_score, axis = 1)
    training_std_deviation = std(training_score, axis = 1)
    testing_std_deviation = std(testing_score, axis = 1)
    testing_mean = mean(testing_score, axis = 1 )

    ## we have got the estimator in this case the perceptron running in 10 fold validation with 
    ## equal division of sizes betwwen .1 and 1. After execution, we get the number of training sizes used, 
    ## the training scores for those sizes and the test scores for those sizes. we will plot a scatter plot 
    ## to see the accuracy results and check for bias vs variance

    # training_size : essentially 10 sets of say a1, a2, a3,,...a10 sizes (this comes from train_size parameter, here we have given linespace for equal distribution betwwen 0.1 and 1 for 10 such values)
    # training_score : training score for the a1 samples, a2 samples...a10 samples, each samples run 10 times since cv value is 10
    # testing_score : testing score for the a1 samples, a2 samples...a10 samples, each samples run 10 times since cv value is 10
    ## the mean and std deviation for each are calculated simply to show ranges in the graph

    plot.plot(training_size, training_mean, label= "Training Data", marker= '+', color = 'blue', markersize = 8)
    plot.fill_between(training_size, training_mean+ training_std_deviation, training_mean-training_std_deviation, color='blue', alpha =0.12 )

    plot.plot(training_size, testing_mean, label= "Testing/Validation Data", marker= '*', color = 'green', markersize = 8)
    plot.fill_between(training_size, testing_mean+ training_std_deviation, testing_mean-training_std_deviation, color='green', alpha =0.14 )

    plot.title("Scoring of our training and testing data vs sample sizes")
    plot.xlabel("Number of Samples")
    plot.ylabel("Accuracy")
    plot.legend(loc= 'best')
    plot.show()
def runGridSearchAndPredict(pipeline, x_train, y_train, x_test, y_test, param_grid, n_jobs = 1, cv = 10, score = 'accuracy'):
    
    response = {}
    training_timer       = CodeTimer('training')
    testing_timer        = CodeTimer('testing')
    learning_curve_timer = CodeTimer('learning_curve')
    predict_proba_timer  = CodeTimer('predict_proba')
    
    with training_timer:
        gridsearch = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv = cv, n_jobs = n_jobs, scoring = score)

        search = gridsearch.fit(x_train,y_train)

        print("Grid Search Best parameters ", search.best_params_)
        print("Grid Search Best score ", search.best_score_)
            
    with testing_timer:
        y_prediction = gridsearch.predict(x_test)
            
#     print("Accuracy score %s" %accuracy_score(y_test,y_prediction))
    print("F1 score %s" %f1_score(y_test,y_prediction, average ='weighted'))
    print("Classification report  \n %s" %(classification_report(y_test, y_prediction)))
    
    with learning_curve_timer:
        plotLearningCurve(_x_train, _y_train, search.best_estimator_)
        
    with predict_proba_timer:
        if hasattr(gridsearch.best_estimator_, 'predict_proba'):
            
            y_probability = gridsearch.predict_proba(x_test)
#             false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_probability[:,1])
#             response['roc_auc_score'] = roc_auc_score(y_test, y_probability[:,1])
#             response['roc_curve'] = (false_positive_rate, true_positive_rate)
    
        else: ## eg SVM, Perceptron doesnt have predict_proba method
            
            response['roc_auc_score'] = 0
            response['roc_curve'] = None
    
    response['learning_curve_time'] = learning_curve_timer.took
    response['testing_time'] = testing_timer.took
    response['_y_prediction'] = y_prediction
#     response['accuracy_score'] = accuracy_score(y_test,y_prediction)
    response['training_time'] = training_timer.took
    response['f1_score']  = f1_score(y_test, y_prediction, average ='weighted')
    response['best_estimator'] = search.best_estimator_
    
    
    return response

In [15]:
_x_train,_x_test,_y_train,_y_test = train_test_split(x,y, test_size = 0.3, stratify= y, random_state =12)

In [16]:
print("label counts in y train %s" %_y_train['y'].value_counts())
print("label counts in y test %s" %_y_test['y'].value_counts())

label counts in y train 0    24010
1     3512
2     1973
3      566
4      225
Name: y, dtype: int64
label counts in y test 0    10290
1     1506
2      845
3      243
4       96
Name: y, dtype: int64


In [ ]:
timer = CodeTimer(name='overalltime')
model_metrics = {}

with timer:
    for model, model_name, model_param_grid in zip(classifiers, classifier_names, classifier_param_grid):

        pipeline = Pipeline([
                ('scaler', StandardScaler()),
                (model_name, model)
        ])

        cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 12)
        result = runGridSearchAndPredict(pipeline, _x_train, _y_train, _x_test, _y_test, model_param_grid , cv = cv,score = 'f1_weighted')

        _y_prediction = result['_y_prediction']

        _matrix = confusion_matrix(y_true = _y_test ,y_pred = _y_prediction, labels = _y_test['y'].values)

        model_metrics[model_name] = {}
        model_metrics[model_name]['confusion_matrix'] = _matrix
        model_metrics[model_name]['training_time'] = result['training_time']
        model_metrics[model_name]['testing_time'] = result['testing_time']
        model_metrics[model_name]['learning_curve_time'] = result['learning_curve_time']
#         model_metrics[model_name]['accuracy_score'] = result['accuracy_score']
        model_metrics[model_name]['f1_score'] = result['f1_score']
#         model_metrics[model_name]['roc_auc_score'] = result['roc_auc_score']
#         model_metrics[model_name]['roc_curve'] = result['roc_curve']
        model_metrics[model_name]['best_estimator'] = result['best_estimator']
        
        
print(timer.took)


Grid Search Best parameters  {}
Grid Search Best score  0.6976891122934457
Code block 'training' took(HH:MM:SS): 1:15:44.573396
Code block 'testing' took(HH:MM:SS): 0:00:32.142927
F1 score 0.7002856261987017
Classification report  
              precision    recall  f1-score   support

          0       0.81      0.87      0.84     10290
          1       0.21      0.14      0.17      1506
          2       0.24      0.19      0.21       845
          3       0.07      0.04      0.05       243
          4       0.05      0.02      0.03        96

avg / total       0.68      0.72      0.70     12980



#### There are some observations we can make: 

#### 1. DecisionTreeClassifier gives perfect predictions gives  best f1 score. And we have similar scores for testing and training data , suggesting less variations/overfitting


#### We will use both these classifiers to make a prediction on the train set

In [32]:
# model_estimates = panda.DataFrame(model_metrics).transpose()

model_metrics

{}

In [30]:
decision_tree_model =model_metrics['decisiontreeclassifier']['best_estimator']

# k_neighbor_model =model_metrics['kneighborsclassifier']['best_estimator']

KeyError: 'decisiontreeclassifier'

In [ ]:
k_neighbor_model

In [ ]:
decision_tree_model

#### At this point our models are ready using which we can predict on our test data. Note since we used  a pipeline, model has the built in scaling.

### Prepare the test data and run prediction


#### remember to check for column dimension

In [ ]:
test_data = panda.read_csv(test_data_path)
test_data.shape

In [45]:
x.shape

(43266, 57)

In [ ]:
test_data.head()

In [ ]:
test_x = test_data[[i for i in test_data.columns.tolist() if i not in columns_not_required]]

In [ ]:
test_x.shape, x.shape

In [49]:
set(x.columns.tolist()) - set(test_x.columns.tolist())

set()

In [56]:

from sklearn.preprocessing import StandardScaler
scaled_x = StandardScaler().fit_transform(test_x)

In [65]:
y_predict = decision_tree_model.predict(test_x)

In [67]:
np.bincount(y_predict)

array([    0, 18208,   125,   196,    14], dtype=int64)

In [68]:
submission_data = panda.read_csv(sample_submission_path)

submission_data.head()

,Complaint-ID,Complaint-Status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation


In [83]:
#lets apply encoding on our target columns

target_dict = {
    
    'Closed with explanation':0,
    'Closed with non-monetary relief':1,
    'Closed with monetary relief':2,
    'Closed':3,
    'Untimely response':4
    
}


target_dict_reverse = {
    
    0:'Closed with explanation',
    1:'Closed with non-monetary relief',
    2:'Closed with monetary relief',
    3:'Closed',
    4:'Untimely response'
    
}
predict = panda.DataFrame({'y':y_predict})
predict.head(45)

,y
0,1
1,1
2,1
3,1
4,1
5,2
6,1
7,1
8,1
9,1


In [84]:
predict['Complaint-Status'] = predict.y.apply(lambda x:target_dict_reverse.get(x))

In [85]:
predict.head()

,y,Complaint-Status
0,1,Closed with explanation
1,1,Closed with explanation
2,1,Closed with explanation
3,1,Closed with explanation
4,1,Closed with explanation


In [89]:
predict['Complaint-ID'] = test_data['complaint_id']

predict.drop(['y'], axis=1, inplace=True)
predict.head(25)

,Complaint-Status,Complaint-ID
0,Closed with explanation,Te-1
1,Closed with explanation,Te-2
2,Closed with explanation,Te-3
3,Closed with explanation,Te-4
4,Closed with explanation,Te-5
5,Closed with non-monetary relief,Te-6
6,Closed with explanation,Te-7
7,Closed with explanation,Te-8
8,Closed with explanation,Te-9
9,Closed with explanation,Te-10


In [90]:
predict[['Complaint-ID','Complaint-Status']].to_csv('dataset/submission_round2.csv', index = False)